# spaCy Named Entity Annotation using Regular Expressions

(C) 2024-2025 by [Damir Cavar](http://damir.cavar.me/)

**Version:** 0.2, November 2025

**Download:** This and various other Jupyter notebooks are available from my [GitHub repo](https://github.com/dcavar/python-tutorial-for-ipython).

The following code shows how simple regular expression matches can be converted to spaCy Named Entity labels. The goal is to provide a first level annotation of entities, manually correct the annotations, and generate a corpus for training neural annotation models.

In [1]:
from __future__ import unicode_literals, print_function
import spacy
from spacy.util import minibatch, compounding
from spacy.training.example import Example
import re
import random
from pathlib import Path

In [2]:
nlp = spacy.load("en_core_web_trf")

c:\Users\damir\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\damir\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\damir\AppData\Local\Programs\Python\Python312\Lib\site-packages\thinc\shims\pytorch.py:261: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for 

In [3]:
banks = """
Bank of America
Bank of America Corporation
Citigroup
Citigroup Inc.
Citi
Goldman Sachs
The Goldman Sachs Group, Inc.
Goldman Sachs Group, Inc.
JP Morgan
JPMorgan Chase
JPMorgan Chase & Co.
JPMorganChase
Morgan Stanley
The PNC Financial Services Group, Inc.
PNC Financial Services Group, Inc.
PNC Bank
U.S. Bancorp
Wells Fargo
Wells Fargo & Company
"""

In [4]:
insurances = """
Farmers Insurance Group
Farmers
Acuity Insurance
Aflac
Aflac Incorporated
Allianz Life
Allied Insurance
Allstate
The Allstate Corporation
American Automobile Association
AAA
American Family Insurance
American Income Life Insurance Company
AIL
American International Group
AIG
Government Employees Insurance Company
GEICO
Liberty Mutual
Liberty Mutual Insurance Company
Zurich Insurance Group
Zurich Insurance Group Ltd
"""

We can filter out regular expression operators and convert them to symbols, as here for the period:

In [5]:
banks = banks.replace(r'.', r'\.')
insurances = insurances.replace(r'.', r'\.')

We can remove empty elements from the list by filtering in the list comprehension. Duplicates are removed by converting the list to a set.

In [6]:
banks_list = { x for x in banks.splitlines() if x }
insurances_list = { x for x in insurances.splitlines() if x }

In [7]:
print(banks_list)

{'Citigroup Inc\\.', 'JPMorganChase', 'PNC Bank', 'Wells Fargo', 'Citigroup', 'PNC Financial Services Group, Inc\\.', 'The PNC Financial Services Group, Inc\\.', 'Goldman Sachs Group, Inc\\.', 'Citi', 'Bank of America Corporation', 'Wells Fargo & Company', 'Morgan Stanley', 'JPMorgan Chase', 'Bank of America', 'JP Morgan', 'U\\.S\\. Bancorp', 'Goldman Sachs', 'JPMorgan Chase & Co\\.', 'The Goldman Sachs Group, Inc\\.'}


The lists can now be converted to labeled groups in Python regular expressions. We will use the labels as Named Entity tags:

In [8]:

regular_expression = re.compile( r"|".join( (r"(?P<BANK>" + r"|".join( banks_list ) + r")", 
											 r"(?P<INSURANCE>" + r"|".join( insurances_list ) + r")") ) )

Here is some sample text that we want to annotate:

In [9]:
sample_text = """
Zurich Insurance Group Ltd is a Swiss insurance company, headquartered in Zürich, and the country's largest insurer.
Wells Fargo is an American multinational financial services company with a significant global presence.
JPMorgan Chase & Co. (stylized as JPMorganChase) is an American multinational financial services firm headquartered in New York City and incorporated in Delaware. It is the largest bank in the United States and the world's largest bank by market capitalization as of 2023.
"""

In [10]:
annotations = []
for match in regular_expression.finditer(sample_text):
	for label, value in match.groupdict().items():
		if value:
			break
	print(f"{label}: {match.start()} {match.end()} {match.group(0)}")
	if label:
		annotations.append( (match.start(), match.end(), label) )

INSURANCE: 1 27 Zurich Insurance Group Ltd
BANK: 118 129 Wells Fargo
BANK: 222 236 JPMorgan Chase
BANK: 256 269 JPMorganChase


We can also run the text through the spaCy NLP pipeline:

In [11]:
doc = nlp(sample_text.strip())

Now we can process sentence by sentence and generate the entity annotations for the training data:

In [12]:
training_data = []
for sentence in doc.sents:
	annotations = []
	for match in regular_expression.finditer(sentence.text):
		for label, value in match.groupdict().items():
			if value:
				break
		if label:
			annotations.append( (match.start(), match.end(), label) )
	if annotations:
		training_data.append( (sentence.text, { 'entities': annotations }) )

The resulting data structure is a list of tuples. The first element is the sentence text. The second is a dictionary with a key `entities` that has a list of antity annotation tuples as value:

In [13]:
for x in training_data:
	print(x[0])
	print(x[1])

Zurich Insurance Group Ltd is a Swiss insurance company, headquartered in Zürich, and the country's largest insurer.
{'entities': [(0, 26, 'INSURANCE')]}

Wells Fargo is an American multinational financial services company with a significant global presence.
{'entities': [(1, 12, 'BANK')]}
JPMorgan Chase & Co. (stylized as JPMorganChase) is an American multinational financial services firm headquartered in New York City and incorporated in Delaware.
{'entities': [(0, 14, 'BANK'), (34, 47, 'BANK')]}


In [14]:
nlp_new = spacy.blank("xx")  # create blank Language class
nlp_new.add_pipe('sentencizer')
ner = nlp_new.add_pipe("ner", last=True)

In [15]:
for _, annotations in training_data:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

In [16]:
nlp_new.begin_training()

In [17]:
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
other_pipes = [pipe for pipe in nlp_new.pipe_names if pipe not in pipe_exceptions]

In [18]:
with nlp_new.disable_pipes(*other_pipes):  # only train NER
    for itn in range(100):
        random.shuffle(training_data)
        losses = {}
        # batch up the examples using spaCy's minibatch
        batches = minibatch(training_data, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            for text, annotations in batch:
                print(text)
                print(annotations)
                doc = nlp_new.make_doc(text)
                example = Example.from_dict(doc, annotations)
                nlp_new.update([example],
                    drop=0.5,  # dropout - make it harder to memorise data
                    losses=losses,
                )
        print("Losses", losses)


Wells Fargo is an American multinational financial services company with a significant global presence.
{'entities': [(1, 12, 'BANK')]}
JPMorgan Chase & Co. (stylized as JPMorganChase) is an American multinational financial services firm headquartered in New York City and incorporated in Delaware.
{'entities': [(0, 14, 'BANK'), (34, 47, 'BANK')]}
Zurich Insurance Group Ltd is a Swiss insurance company, headquartered in Zürich, and the country's largest insurer.
{'entities': [(0, 26, 'INSURANCE')]}
Losses {'ner': 55.922169387340546}
Zurich Insurance Group Ltd is a Swiss insurance company, headquartered in Zürich, and the country's largest insurer.
{'entities': [(0, 26, 'INSURANCE')]}
JPMorgan Chase & Co. (stylized as JPMorganChase) is an American multinational financial services firm headquartered in New York City and incorporated in Delaware.
{'entities': [(0, 14, 'BANK'), (34, 47, 'BANK')]}

Wells Fargo is an American multinational financial services company with a significant global

In [19]:
for text, _ in training_data:
    doc = nlp_new(text)
    print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
    print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

Entities [('JPMorgan Chase', 'BANK'), ('JPMorganChase', 'BANK')]
Tokens [('JPMorgan', 'BANK', 3), ('Chase', 'BANK', 1), ('&', '', 2), ('Co', '', 2), ('.', '', 2), ('(', '', 2), ('stylized', '', 2), ('as', '', 2), ('JPMorganChase', 'BANK', 3), (')', '', 2), ('is', '', 2), ('an', '', 2), ('American', '', 2), ('multinational', '', 2), ('financial', '', 2), ('services', '', 2), ('firm', '', 2), ('headquartered', '', 2), ('in', '', 2), ('New', '', 2), ('York', '', 2), ('City', '', 2), ('and', '', 2), ('incorporated', '', 2), ('in', '', 2), ('Delaware', '', 2), ('.', '', 2)]
Entities [('Wells Fargo', 'BANK')]
Tokens [('\n', '', 2), ('Wells', 'BANK', 3), ('Fargo', 'BANK', 1), ('is', '', 2), ('an', '', 2), ('American', '', 2), ('multinational', '', 2), ('financial', '', 2), ('services', '', 2), ('company', '', 2), ('with', '', 2), ('a', '', 2), ('significant', '', 2), ('global', '', 2), ('presence', '', 2), ('.', '', 2)]
Entities [('Zurich Insurance Group Ltd', 'INSURANCE')]
Tokens [('Zurich',

In [20]:
output_dir = Path("./models_ner/")

In [28]:
if not output_dir.exists():
    output_dir.mkdir()
nlp_new.to_disk(output_dir)

In [29]:
nlp_test = spacy.load(output_dir)

In [30]:
for text, _ in training_data:
    doc = nlp_test(text)
    print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
    #print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

Entities [('Zurich Insurance Group', 'INSURANCE')]
Entities [('Wells Fargo', 'BANK')]
Entities [('JPMorgan Chase & Co.', 'BANK'), ('JPMorganChase', 'BANK')]


**(C) 2024-2025 by [Damir Cavar](http://damir.cavar.me/) <<dcavar@iu.edu>>**